In [1]:
import os

In [2]:
%pwd

'c:\\Users\\hp\\OneDrive\\Bureau\\Forecast Pipeline\\Demand-Forecasting-Pipeline\\Teraflow-Forecast-Pipeline\\steps'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\hp\\OneDrive\\Bureau\\Forecast Pipeline\\Demand-Forecasting-Pipeline\\Teraflow-Forecast-Pipeline'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig_LSTM:
    root_dir: Path
    model_dir_name: str
    X_train_data_path : Path
    y_train_data_path : Path
    X_test_data_path : Path
    y_test_data_path : Path
    model_name : str
    x_scaler_name : str
    y_scaler_name : str
    runtime_name : str
    horizon : int
    early_stopping : int
    epochs : int
    learning_rate : int

In [6]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH ,
        params_fileopath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_fileopath)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config_LSTM(self) -> ModelTrainerConfig_LSTM:
        config = self.config.model_trainer_lstm
        params = self.params.horizon
        cnn_params = self.params.LSTM
        
        create_directories([config.root_dir])
        create_directories([os.path.join(config.root_dir,config.model_dir_name)])
        model_trainer_config = ModelTrainerConfig_LSTM(
            root_dir = config.root_dir,
            model_dir_name = config.model_dir_name,
            X_train_data_path=config.X_train_data_path,
            y_train_data_path=config.y_train_data_path,
            X_test_data_path= config.X_test_data_path,
            y_test_data_path= config.y_test_data_path,
            model_name = config.model_name,
            x_scaler_name = config.x_scaler_name,
            y_scaler_name = config.y_scaler_name,
            horizon = params.h,
            runtime_name = config.runtime_name,
            early_stopping= cnn_params.early_stopping,
            epochs= cnn_params.epochs,
            learning_rate= cnn_params.lr
            
        )
        return model_trainer_config

In [8]:
import pandas as pd
import os
from src.mlProject.utils import logger
import joblib
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import time
import pickle

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras.metrics import RootMeanSquaredError

In [9]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig_LSTM):
        self.config = config
        
    def load_scale_data(self):
        X_train =np.array(pd.read_csv(self.config.X_train_data_path))
        y_train =np.array(pd.read_csv(self.config.y_train_data_path))
        X_test = np.array(pd.read_csv(self.config.X_test_data_path))
        y_test = np.array(pd.read_csv(self.config.y_test_data_path))
        
        std_x = StandardScaler()
        X_train_scaled = std_x.fit_transform(X_train)
        X_test_scaled = std_x.transform(X_test)

        std_y = StandardScaler()
        y_train_scaled = std_y.fit_transform(y_train)
        y_test_scaled = std_y.transform(y_test)
        
        np.savetxt(os.path.join(os.path.join(self.config.root_dir,self.config.model_dir_name),"X_test_scaled.csv"), X_test_scaled, delimiter=',')
        np.savetxt(os.path.join(os.path.join(self.config.root_dir,self.config.model_dir_name),"y_test_scaled.csv"), y_test_scaled, delimiter=',')
        
        joblib.dump(std_x,os.path.join(os.path.join(self.config.root_dir,self.config.model_dir_name),self.config.x_scaler_name))
        joblib.dump(std_y,os.path.join(os.path.join(self.config.root_dir,self.config.model_dir_name),self.config.y_scaler_name))

        
        return X_train_scaled,y_train_scaled
    
    def create_model(self,past_time_steps, future_time_steps):
        
        model = Sequential([layers.Input((past_time_steps+2,1)),
                            layers.LSTM(128),
                            layers.Dense(64, activation='relu'),
                            layers.Dense(future_time_steps, activation='linear')])

        
        optimizer = Adam(learning_rate=self.config.learning_rate)
        model.compile(loss='mse', optimizer=optimizer, metrics=[RootMeanSquaredError()])

        return model
    
    def train(self,X_train_scaled,y_train_scaled):    
        
        X_train, X_val, y_train, y_val = train_test_split(X_train_scaled, y_train_scaled, test_size=0.2, random_state=42)
        model = self.create_model(self.config.horizon,self.config.horizon)
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=self.config.early_stopping,mode='min')

        start_lstm = time.time()
        hist = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=self.config.epochs,callbacks=[early_stopping])
        end_lstm = time.time()
        
        training_time = end_lstm - start_lstm
        training_time = {"training time" : training_time}
        save_json(path=Path(os.path.join(os.path.join(self.config.root_dir,self.config.model_dir_name),self.config.runtime_name)),data=training_time)        
        model.save(os.path.join(os.path.join(self.config.root_dir,self.config.model_dir_name),self.config.model_name))
        
        history_path = os.path.join(os.path.join(self.config.root_dir,self.config.model_dir_name),'training_history.pkl')

        with open(history_path, 'wb') as file:
            pickle.dump(hist.history, file)

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config_LSTM()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    X_train_scaled,y_train_scaled = model_trainer_config.load_scale_data()
    model_trainer_config.train(X_train_scaled,y_train_scaled)
except Exception as e:
    raise e

[2024-04-18 09:17:12,322: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-18 09:17:12,331: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-18 09:17:12,334: INFO: common: created directory at: artifacts]
[2024-04-18 09:17:12,337: INFO: common: created directory at: artifacts/model_trainer]
[2024-04-18 09:17:12,339: INFO: common: created directory at: artifacts/model_trainer\LSTM]
Epoch 1/2
5/5 [==============================] - 8s 688ms/step - loss: 1.0856 - root_mean_squared_error: 1.0419 - val_loss: 0.5998 - val_root_mean_squared_error: 0.7745
Epoch 2/2
5/5 [==============================] - 2s 476ms/step - loss: 1.0030 - root_mean_squared_error: 1.0015 - val_loss: 0.5203 - val_root_mean_squared_error: 0.7213
[2024-04-18 09:17:24,760: INFO: common: json file saved at: artifacts\model_trainer\LSTM\lstm_runtime.json]


c:\Users\hp\anaconda3\envs\mlflowEnv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
